In [ ]:
import os
import librosa
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
import tensorflow as tf
import keras
from keras import layers, models ,Model ,Input
from keras.layers import Dense, Flatten, Activation, BatchNormalization
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/VoiceRecognition/main')
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/gdrive/MyDrive/VoiceRecognition/main']


In [ ]:
from training1 import *

ModuleNotFoundError: ignored

In [ ]:
def newData_learn(model_dir, record_dir):
  record = record_dir
  base_model = models.load_model(model_dir)
  record_list = os.listdir(record) 

  record_data = []
  record_target = []
   
  for i, (root, dirs, files) in enumerate (os.walk(record)):
    for file in files :
      if '.wav'  not in file in file :
        continue
      else :
        audio_path = os.path.join(root,file)
        mfcc, delta = MFCC(audio_path)
        mfcc = mfcc.astype(float)
        dirname = os.path.dirname(audio_path) 
        record_data.append(mfcc)
        record_target.append(dirname)

  padding(record_data,base_model.input)
  record_data = np.array(record_data)

  re_encoder = LabelEncoder()
  re_encoder.fit(record_target)
  retarget_encoded = re_encoder.transform(record_target)
  print(record_target, '==>', retarget_encoded)

  record_target = tf.keras.utils.to_categorical(np.array(retarget_encoded))

  rectrain_x, recval_x, rectrain_y, recval_y = train_test_split(record_data, record_target, test_size=0.3, shuffle=True, stratify=record_target, random_state=42)
  rectrain_x = np.expand_dims(rectrain_x, -1)
  recval_x = np.expand_dims(recval_x, -1)
  print("record train size : ",rectrain_x.shape[0])

  net = Dense(len(record_list), activation='softmax' ,name="dense_a")(base_model.layers[-2].output)
  model2 = Model(inputs = base_model.input, outputs = net)

  model2.summary()

  model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

  model2.fit(rectrain_x,
            rectrain_y,
            epochs=50,
            batch_size=32,
            verbose=1,
            validation_data=(recval_x, recval_y),
            )
  
  return model2


In [ ]:
def pred_data(test_dir, input_shape):
  test_data = []
  
  for i, (root, dirs, files) in enumerate (os.walk(test_dir)):
    for file in files :
      if '.wav'  not in file in file :
        continue
      else :
        audio_path = os.path.join(root,file)
        mfcc, delta = MFCC(audio_path)
        mfcc = mfcc.astype(float)
        test_data.append(mfcc)

  padding(test_data,input_shape)
  test_data = np.array(test_data)

  test_data = np.expand_dims(test_data, -1)

  return test_data

In [ ]:
model3 = newData_learn('/content/gdrive/MyDrive/model/best_model.h5','/content/gdrive/MyDrive/VoiceRecognition/data/Recording/')

OSError: ignored

In [ ]:
test_x = pred_data('/content/gdrive/MyDrive/VoiceRecognition/data/Recording/zzds0623/',(20,262,1))
pred = np.argmax(model3.predict(test_x), axis=-1)
print(pred)

NameError: ignored